# BLP: OLS estimates

We are given data on OTC headache medicine sales at the store x week level, covering 3 brands and 3 package sizes. Our sales variables include: 

-  Count - Number of People that go into the store each week.
-  Promotion - Indicator variable for promotion on the product that week
-  Price - Price of the package
-  Week - time indicator
-  Store - market indicator.


Our demographic variables include:

- Income - Household Income for person.
- Week - time indicator 
- Store - market indicator

Finally, our instruments include:
- Cost - wholesale cost
- Prices at all stores

We first estimate the following model using OLS (1-3) and with our instruments:

$$ u_{ijt} = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \epsilon_{ijt} $$

where $\epsilon_{ijt}$ is iid logit. We can use the transformation in class to render this model estimable. In particular, the logit distribution of the error term implies that:

$$ Pr(U_{ij} \geq U_{ik}) = \frac{\exp(X_{jt}\beta + \alpha p_{jt} + \xi_{jt})}{\sum_{j=1}^{J}\exp(X_{jt}\beta + \alpha p_{jt} + \xi_{jt})} $$

We can thus take logs of shares and obtain the estimating equation:

$$ \log(s_{jt}) - \log(s_{0t}) = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} $$

where $s_{0t}$ is the share of customers who decide to buy nothing in period $t$. We prepare the data by computing shares before estimating our regression equation. The following code blocks display both these processes.

## Preparing data

In [92]:
global root "."
global data "$root/PS1_Data"
import delimited "$data/OTC_Data.csv", clear




(encoding automatically selected: ISO-8859-1)
(8 vars, 38,544 obs)


In [93]:
list in 1/4


     +----------------------------------------------------------------+
     | store   week   brand   sales_   count   price_   prom_   cost_ |
     |----------------------------------------------------------------|
  1. |     2      1       1       16   14181     3.29       0    2.06 |
  2. |     2      2       1       12   13965     3.27       0    2.04 |
  3. |     2      3       1        6   13538     3.37       0    2.15 |
  4. |     2      4       1       12   13735      3.3       0    2.07 |
     +----------------------------------------------------------------+


In [94]:
sum store week brand


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
       store |     38,544    74.76712    33.34526          2        123
        week |     38,544        24.5    13.85358          1         48
       brand |     38,544           6    3.162319          1         11


In [95]:
// Normalize prices to per 50 tablets
replace price_ = price_*2 if (brand == 1) | (brand == 4) | (brand == 7)
replace price_ = price_/2 if brand == 3 | brand == 6 | brand == 9 | brand == 11


(10,512 real changes made)

(14,016 real changes made)


In [96]:
// Get data into store * week level to construct shares
reshape wide sales_ price_ prom_ cost_, i(store week count) j(brand)

(j = 1 2 3 4 5 6 7 8 9 10 11)

Data                               Long   ->   Wide
-----------------------------------------------------------------------------
Number of observations           38,544   ->   3,504       
Number of variables                   8   ->   47          
j variable (11 values)            brand   ->   (dropped)
xij variables:
                                 sales_   ->   sales_1 sales_2 ... sales_11
                                 price_   ->   price_1 price_2 ... price_11
                                  prom_   ->   prom_1 prom_2 ... prom_11
                                  cost_   ->   cost_1 cost_2 ... cost_11
-----------------------------------------------------------------------------


In [97]:
// Construct shares: tally outside option and compute log shares
gen buy = 0
forvalues i = 1/11 {
    gen share`i' = sales_`i'/count
    gen lshare`i' = log(share`i')
    qui replace buy = buy + sales_`i'
}
gen nobuy = count-buy
gen lshare0 = log(nobuy/count)

// Subtract logs to get LHS
forvalues i = 1/11 {
    gen y`i' = lshare`i' - lshare0
}

// Return to store-week * brand level
drop sales* lshare* *buy*
reshape long price_ prom_ cost_ share y, i(store week count) j(brand)








(j = 1 2 3 4 5 6 7 8 9 10 11)

Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations            3,504   ->   38,544      
Number of variables                  58   ->   9           
j variable (11 values)                    ->   brand
xij variables:
           price_1 price_2 ... price_11   ->   price_
              prom_1 prom_2 ... prom_11   ->   prom_
              cost_1 cost_2 ... cost_11   ->   cost_
              share1 share2 ... share11   ->   share
                          y1 y2 ... y11   ->   y
-----------------------------------------------------------------------------


In [98]:
list in 1/4


     +------------------------------------------------------------------+
  1. | store | week | count | brand | price_ | prom_ | cost_ |    share |
     |     2 |    1 | 14181 |     1 |   6.58 |     0 |  2.06 | .0011283 |
     |------------------------------------------------------------------|
     |                                    y                             |
     |                            -6.781129                             |
     +------------------------------------------------------------------+

     +------------------------------------------------------------------+
  2. | store | week | count | brand | price_ | prom_ | cost_ |    share |
     |     2 |    1 | 14181 |     2 |   4.82 |     0 |  3.43 | .0007757 |
     |------------------------------------------------------------------|
     |                                    y                             |
     |                            -7.155822                             |
     +------------------------------

In [99]:
// Construct store-week variable for brand FE regressions
tostring store, gen(store_id)
tostring week, gen(week_id)
gen store_week_id = store_id + " " + week_id
encode store_week_id, gen(t)

// Construct store-brand variable for store-brand FE regressions
tostring brand, gen(brand_id)
gen store_brand_id = store_id + " " + brand_id
encode store_brand_id, gen(sb)


store_id generated as str3

week_id generated as str2



brand_id generated as str2




In [100]:
label variable price "Price"
label variable prom_ "Promotion"

Now that we have our data organized at the store-week * brand level, we can carry out our regression analyses. We will report all the Stata estimates in a single table at the end.

In [101]:
// 1. Use price and promotion as product characteristics
qui reg y price prom_
estimates store model1ols
estadd local spec "OLS"
estadd local FE "No"

// 2. Price and promotion plus brand dummies
qui xtset brand t
qui xtreg y price prom_, fe vce(robust)
estimates store model2ols
estadd local spec "OLS"
estadd local FE "Brand"

// 3. Price and promotion plus store-brand dummies
qui xtset sb week
qui xtreg y price prom_, fe vce(robust)
estimates store model3ols
estadd local spec "OLS"
estadd local FE "Store-Brand"

// 4. Now we use wholesale cost as an instrument for price, and estimate the same models as 1-3
qui ivreg y prom_ (price=cost_), robust
estimates store model1costiv
estadd local spec "Cost IV"
estadd local FE "No"

qui xtset brand t
qui xtivreg y prom_ (price=cost_), fe vce(robust)
estimates store model2costiv
estadd local spec "Cost IV"
estadd local FE "Brand"

qui xtset sb week
qui xtivreg y prom_ (price=cost_), fe vce(robust)
estimates store model3costiv
estadd local spec "Cost IV"
estadd local FE "Store-Brand"



save "$data/reg_data", replace





added macro:
               e(spec) : "OLS"


added macro:
                 e(FE) : "No"





added macro:
               e(spec) : "OLS"


added macro:
                 e(FE) : "Brand"





added macro:
               e(spec) : "OLS"


added macro:
                 e(FE) : "Store-Brand"




added macro:
               e(spec) : "Cost IV"


added macro:
                 e(FE) : "No"





added macro:
               e(spec) : "Cost IV"


added macro:
                 e(FE) : "Brand"





added macro:
               e(spec) : "Cost IV"


added macro:
                 e(FE) : "Store-Brand"

file ./PS1_Data/reg_data.dta saved


In [102]:
// 5. Next, we use a Hausman instrument for price, and estimate the models from 1-3
// First, construct the instrument:

import delimited "$data/OTCDataInstruments.csv", clear
forvalues i = 1/30 {
    qui replace pricestore`i' = 0 if store == `i'
}

// Compute average of all other prices
gen hausman = 0
forvalues i = 1/30 {
    qui replace hausman = hausman + pricestore`i'
}
replace hausman = hausman/29
keep store week brand hausman

// Add average price to our regressors
qui merge 1:1 store week brand using "$data/reg_data.dta"
drop _merge


(encoding automatically selected: ISO-8859-2)
(35 vars, 38,544 obs)




(38,544 real changes made)





In [103]:
// 5. Estimate models
qui ivreg y prom_ (price=hausman)
estimates store model1hausmaniv
estadd local spec "Hausman IV"
estadd local FE "No"

qui xtset brand t
qui xtivreg y prom_ (price=hausman), fe vce(robust)
estimates store model2hausmaniv
estadd local spec "Hausman IV"
estadd local FE "Brand"

qui xtset sb week
qui xtivreg y prom_ (price=hausman), fe vce(robust)
estimates store model3hausmaniv
estadd local spec "Hausman IV"
estadd local FE "Store-Brand"






added macro:
               e(spec) : "Hausman IV"


added macro:
                 e(FE) : "No"





added macro:
               e(spec) : "Hausman IV"


added macro:
                 e(FE) : "Brand"





added macro:
               e(spec) : "Hausman IV"


added macro:
                 e(FE) : "Store-Brand"


In [104]:
esttab model1ols model1costiv model1hausmaniv model2ols model2costiv model2hausmaniv model3ols model3costiv model3hausmaniv using "$data/estimates.tex",  scalars("spec Model" "FE FE") nomtitles se replace

(output written to ./PS1_Data/estimates.tex)


In [105]:
keep store week brand price share
reshape wide price share, i(store week) j(brand)
merge 1:m store week using "$data/reg_data"



(j = 1 2 3 4 5 6 7 8 9 10 11)

Data                               Long   ->   Wide
-----------------------------------------------------------------------------
Number of observations           38,544   ->   3,504       
Number of variables                   5   ->   24          
j variable (11 values)            brand   ->   (dropped)
xij variables:
                                 price_   ->   price_1 price_2 ... price_11
                                  share   ->   share1 share2 ... share11
-----------------------------------------------------------------------------


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                            38,544  (_merge==3)
    -----------------------------------------


In [106]:
keep store week brand price* share*

In [107]:
save "$data/tmp", replace

(file ./PS1_Data/tmp.dta not found)
file ./PS1_Data/tmp.dta saved


In [109]:
local allmodels "model1ols model1costiv model1hausmaniv model2ols model2costiv model2hausmaniv model3ols model3costiv model3hausmaniv"
disp("A")
foreach m in `allmodels' {
    use "$data/tmp", clear
    estimates restore `m'
    forvalues i = 1/11 {
        local alpha=_b[price_]
        gen e_`i' = `alpha'*price_`i'*share`i'
        replace e_`i' = -`alpha'*price_*(1-share) if brand == `i'
    }
    sort store week brand
    keep if store == 9 & week == 10
    keep e*
    export delimited "./elasticity_`m'.csv", replace
}



A

(results model1ols are active now)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(38,533 observations deleted)
file ./elasticity_model1ols.csv saved
(results model1costiv are active now)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(38,533 observations deleted)
file ./elasticity_model1costiv.csv saved
(results model1hausmaniv are active now)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real changes made)
(3,504 real ch

In [84]:
estimates restore model1ols
local coef1=_b[price_]
disp("`coef1'")


(results model1ols are active now)


.1146063612813487
